In [1]:
import numpy as np
import pandas as pd
import os 
import sys
import sklearn
from sklearn.metrics.pairwise import cosine_similarity 
from datetime import datetime
from aqi_calc import compute_aqi

# Preprocess

In [2]:
!pwd

/tf


In [3]:
dp = 'PRSA_Data_20130301-20170228/'

In [4]:
filelist = os.listdir(dp)
filelist.remove('.DS_Store')

In [5]:
dfs = []

for i in filelist:
    df = pd.read_csv(dp + i)
    dfs.append(df)
    
beijing = pd.concat(dfs, axis=0)
len(beijing)

385704

In [6]:
beijing.isnull().sum()

No             0
year           0
month          0
day            0
hour           0
PM2.5       8043
PM10        5965
SO2         8352
NO2        11362
CO         19404
O3         12199
TEMP         379
PRES         374
DEWP         384
RAIN         371
wd          1743
WSPM         305
station        0
dtype: int64

In [7]:
#Re-index all rows. Drop 'No'. 
#Create datetime object for later time series analysis.

beijing.index = range(len(beijing.index))
beijing.drop('No', axis=1, inplace=True)

beijing['date'] = beijing.apply(lambda x: '{}-{}-{}'.format(x['year'], x['month'], x['day']), axis = 1)
beijing['date'] = beijing['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


def season_values(month):
    
    if (month >= 3) & (month <= 5):
        
        return 'spring'
        
    elif (month >= 6) & (month <= 8):
        
        return 'summer'
        
    elif (month >= 9) & (month <= 11 ):
        
        return 'fall'
        
    else: 
        return 'winter'
    

beijing['Season'] = beijing['month'].apply(lambda x: season_values(x)) 

szn_dums = pd.get_dummies(beijing['Season'])
beijing.drop('Season', axis = 1, inplace = True)
beijing = pd.concat([beijing, szn_dums], axis = 1)


In [8]:
#Group by year and month to fill in null values based on mean for those dates.
y = beijing.groupby(['year', 'month']).mean()

In [ ]:
for i in range(len(beijing)):
    row = beijing.loc[i, :]
    missing_cols = row.index[row.isnull()].tolist()

    if 'wd' in missing_cols:
        missing_cols.remove('wd')
    
    for j in missing_cols: 
        beijing.loc[i, j] = y.loc[(beijing.loc[i, 'year'], beijing.loc[i, 'month']), j]



In [ ]:
beijing.fillna(0, inplace = True)
empty_wd_idxs = beijing[beijing['wd'] == 0].index

for i in list(empty_wd_idxs):
    
    beijing.loc[i, 'wd'] = beijing.loc[i-1, 'wd']



In [ ]:
beijing.columns

In [ ]:
wd_dums = pd.get_dummies(beijing.wd)
beijing.drop('wd', axis = 1, inplace = True)
beijing = pd.concat([beijing, wd_dums], axis = 1)

# Pollutant Unit Conversion

In [ ]:
mw = {'CO': 28, 'O3': 48, 'NO2': 46, 'SO2': 64}

#convert pollutant from ug/m3 to ppb based on temp. Convert to ppm for CO and O3

for i in mw.keys():
    if (i == 'CO') | (i == 'O3'):
        beijing[i] = (beijing['TEMP'] + 273.15)*beijing[i]*0.001/(12.187 * mw[i])
    else:
         beijing[i] = (beijing['TEMP'] + 273.15)*beijing[i]/(12.187 * mw[i])
        


In [14]:
beijing

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,...,NNW,NW,S,SE,SSE,SSW,SW,W,WNW,WSW
0,2013,3,1,0,6.0,18.0,1.751674,33.297565,0.640612,0.041106,...,0,1,0,0,0,0,0,0,0,0
1,2013,3,1,1,6.0,15.0,1.749110,33.248822,0.639675,0.041046,...,0,1,0,0,0,0,0,0,0,0
2,2013,3,1,2,5.0,18.0,13.344805,33.200079,0.558895,0.024219,...,0,0,0,0,0,0,0,0,1,0
3,2013,3,1,3,6.0,20.0,2.093547,33.163521,1.153325,0.026589,...,0,0,0,0,0,0,0,1,0,0
4,2013,3,1,4,5.0,17.0,1.742700,33.126964,0.477998,0.033925,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385699,2017,2,28,19,27.0,72.0,2.907299,46.516780,0.664525,0.007753,...,0,0,0,0,0,0,0,1,0,0
385700,2017,2,28,20,47.0,55.0,6.167112,43.406374,0.912111,0.009190,...,0,1,0,0,0,0,0,0,0,0
385701,2017,2,28,21,18.0,28.0,1.447495,15.104299,0.413570,0.030880,...,0,0,0,0,0,0,0,0,0,0
385702,2017,2,28,22,18.0,20.0,3.233787,16.496998,0.410640,0.028266,...,0,0,0,0,0,0,0,0,1,0


# Beijing Daily: AQI Daily Calculation 

In [15]:
aqi_names = {'PM2.5': 'pm2', 'PM10': 'pm10', 'O3': 'o3'}

for i in aqi_names.keys(): 
    insert_col = 'AQI_' + i
    beijing[insert_col] = beijing[i].apply(lambda x: compute_aqi(x, aqi_names[i]))

beijing['AQI'] = list(beijing.iloc[:, -3:].max(axis = 1))

beijing

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,...,W,WNW,WSW,AQI_PM2.5,AQI_PM10,AQI_CO,AQI_NO2,AQI_SO2,AQI_O3,AQI
0,2013,3,1,0,6.0,18.0,1.751674,33.297565,0.640612,0.041106,...,0,0,0,19.480519,16.666667,7.279686,31.412797,2.502392,38.061074,38.061074
1,2013,3,1,1,6.0,15.0,1.749110,33.248822,0.639675,0.041046,...,0,0,0,19.480519,13.888889,7.269029,31.366813,2.498729,38.005358,38.005358
2,2013,3,1,2,5.0,18.0,13.344805,33.200079,0.558895,0.024219,...,0,1,0,16.233766,16.666667,6.351076,31.320829,19.064007,22.424788,31.320829
3,2013,3,1,3,6.0,20.0,2.093547,33.163521,1.153325,0.026589,...,1,0,0,19.480519,18.518519,13.105964,31.286341,2.990782,24.619395,31.286341
4,2013,3,1,4,5.0,17.0,1.742700,33.126964,0.477998,0.033925,...,0,1,0,16.233766,15.740741,5.431791,31.251853,2.489571,31.411624,31.411624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385699,2017,2,28,19,27.0,72.0,2.907299,46.516780,0.664525,0.007753,...,1,0,0,73.630522,59.414141,7.551425,43.883755,4.153284,7.178515,73.630522
385700,2017,2,28,20,47.0,55.0,6.167112,43.406374,0.912111,0.009190,...,0,0,0,113.791165,51.000000,10.364894,40.949409,8.810160,8.509450,113.791165
385701,2017,2,28,21,18.0,28.0,1.447495,15.104299,0.413570,0.030880,...,0,0,0,55.919679,25.925926,4.699660,14.249338,2.067850,28.592499,55.919679
385702,2017,2,28,22,18.0,20.0,3.233787,16.496998,0.410640,0.028266,...,0,1,0,55.919679,18.518519,4.666359,15.563206,4.619695,26.171935,55.919679


In [16]:
beijing['weekday'] = beijing['date'].dt.dayofweek 
beijing['weekday'] = beijing['weekday'].apply(lambda x: 0 if (x == 5) | (x == 6) else 1)

# Beijing Daily (groupby date)

In [17]:
beijing_daily = beijing.copy()
beijing_daily.drop(['month', 'day'], axis = 1, inplace = True)
beijing_daily = beijing_daily.groupby('date').mean()

# Save Beijing & Beijing Daily

In [18]:
beijing_daily.to_csv('beijing_grouped_daily.csv')

In [19]:
beijing.to_csv('beijing_master.csv', index = False)#beijing with date, mo, yr, day, wd, pollutants, aqis, wd and szn dums